# Corrector Coil

Here we will make a corrector coil as a 3D Cartesian grid and track.



In [ ]:
import numpy as np
from math import pi
from scipy.constants import c
import os

from pmd_beamphysics.fields.corrector_modeling import make_dipole_corrector_fieldmesh

from impact import Impact

# Create FieldMesh

In [ ]:
R = 2 * 2.54e-2  # 2" radius [m]
L = 0.1  # Length along z [m]
theta = np.pi / 2  # Opening angle [rad]
current = 100  # Current [Amp]

FM = make_dipole_corrector_fieldmesh(
    current=current,
    xmin=-R,
    xmax=R,
    nx=21,
    ymin=-R,
    ymax=R,
    ny=21,
    zmin=-5 * L / 2,
    zmax=5 * L / 2,
    nz=101,
    mode="saddle",
    R=R,
    L=L,
    theta=theta,
    npts=20,
    plot_wire=True,
    tilt=pi / 2,
)

In [ ]:
FM.plot_onaxis("Bx")

# Prepare for Impact-T

In [ ]:
FM.write_impact_emfield_cartesian("1T1.T7")

In [ ]:
!head '1T1.T7'

In [ ]:
!ls -ahl

# Run Impact-T

In [ ]:
I = Impact("ImpactT.in")
I.ele["COIL"]  # ['rf_field_scale'] = 1

In [ ]:
FM = I.fieldmaps["1T1.T7"]["field"]
FM.plot_onaxis("Bx")

In [ ]:
I.ele["COIL"]

In [ ]:
I.fieldmaps["1T1.T7"]

In [ ]:
pz0 = 10e6

I.track1(pz0=pz0)

I.plot(["mean_x", "mean_y"])

# Check field integral

In [ ]:
z0 = FM.coord_vec("z")
Bx0 = np.real(FM.Bx[FM.shape[0] // 2, FM.shape[1] // 2, :])

BL = np.trapezoid(Bx0, z0)

BL

In [ ]:
theta0 = -BL * c / pz0

theta0

In [ ]:
I.track1(pz0=pz0)
theta1 = I.particles["final_particles"].py / pz0

theta1

In [ ]:
rel_diff = abs(theta1 / theta0 - 1)
rel_diff

# Check that both are close

In [ ]:
assert rel_diff < 1e-3

# Cleanup

In [ ]:
os.remove("1T1.T7")